## <font color=blue>Init en Settings voor de profile generatorbar</font>


In [1]:
import numpy as np
import pandas as pd
import time

#Deze lijst is om de execute times uit te kunnen rekenen. Alleen voor debug purposes.
timingList = []

numberOfProfilesToGenerate = int(input("Hoeveel profiles wil je genereren? "))

#Hoe meer kieskeurig het profiel, hoe minder fav ingredienten
pickyRange = [1,0]
while pickyRange[0] > pickyRange[1] or (pickyRange[0] < 0 or pickyRange[0] > 1) or (pickyRange[1] < 0 or pickyRange[1] > 1):
    print("De kieskeurigheid geeft aan hoe kieskeurig de gebruiker is. 0 eet alles en 1 eet niks. (float tussen 0 en 1)")
    pickyRange = [float(input("Kieskeurig min?: ")),float(input("Kieskeurig max?: "))] 
    if pickyRange[0] > pickyRange[1] or (pickyRange[0] < 0 or pickyRange[0] > 1) or (pickyRange[1] < 0 or pickyRange[1] > 1):
        print("Je hebt t gewoon fout gedaan... Het is niet erg hoor, T is gewoon dat ik het jammer vind. Probeer het maar ff op nieuw. ")

#Importeer de lijst van ingredienten hier.
timingList.append(time.time()) #Start timer
RecipeIngredients = pd.read_csv("Benodigde Datasets/ingr_pivot_lunch_diner_noten.csv")
listOfIngredients = list(RecipeIngredients)[1:]
timingList.append(time.time()) #Stop Timer

Hoeveel profiles wil je genereren? 1000
De kieskeurigheid geeft aan hoe kieskeurig de gebruiker is. 0 eet alles en 1 eet niks. (float tussen 0 en 1)
Kieskeurig min?: 0.1
Kieskeurig max?: 0.6


## <font color=blue>Function Declaration </font>

In [2]:
def rtnrList(recipeIngredients, pickyness):
    nrlist = []
    for i in range(len(recipeIngredients)):
        if np.random.uniform(0, 1) < pickyness*1: #only 10% of their pickyness is accounted for
            nrlist.append(i)
    return nrlist

def getFavIngredients(recipeIngredients, pickyness):
    #Create Dictionary with False for all ingredients
    #Indices chosen by random
    indices = rtnrList(recipeIngredients, pickyness)
    #Recipes of chosen indices including ingredients
    FavRecipeIng = recipeIngredients.iloc[indices]
    #Ingredients based on fav recipes in one row
    FavIngredients = FavRecipeIng.iloc[:,1:].sum() != 0
    #All fav recipes in one column
    FavRecipes = FavRecipeIng.iloc[:,0]
    
    #Makes a True and False list from RecipeIngredients and FavRecipes.
    AllRecipeNames = recipeIngredients.iloc[:,0].isin(FavRecipes)
    #Concatinates the names and the booleans together
    datafr = pd.concat([recipeIngredients.iloc[:,0],AllRecipeNames],axis=1)
    #Sets Column names
    datafr.columns = ["Recipe","isin"]
    #Transposes dataset and sets index
    datafr = datafr.set_index("Recipe").transpose()

    return FavIngredients,datafr

## <font color=blue>Data generation </font>

In [3]:
#Creeer een dictionary
timingList.append(time.time()) #Start timer
UserIngredients = pd.DataFrame(dict((i, [],) for i in listOfIngredients)).astype(bool)
UserRecipes = pd.DataFrame(columns=RecipeIngredients.iloc[:,0]).astype(bool)
timingList.append(time.time()) #Stop Timer

#Execute de onderstaande code voor elk persoon.
timingList.append(time.time()) #Start timer
for i in range(numberOfProfilesToGenerate):
    pickyness = np.random.uniform(pickyRange[0], pickyRange[1])
    FavIngr, FavRecipes = getFavIngredients(RecipeIngredients,pickyness)
    UserIngredients = pd.concat([UserIngredients,pd.DataFrame(FavIngr).transpose()],axis=0,ignore_index=True)
    UserRecipes = pd.concat([UserRecipes,pd.DataFrame(FavRecipes)],ignore_index=True)
    
timingList.append(time.time()) #Stop Timer


In [4]:
#Convert it into usable datasets (User favs + Specific Recipe Ingredients)
FinalXDataset = pd.DataFrame(columns=list(UserIngredients.columns)+list(UserRecipes.columns))

print(UserIngredients.iloc[0,:])



for i in range(len(UserRecipes)):
    for o in range(len(UserRecipes.iloc[i,:])):
        value = UserRecipes.iloc[i,o]
        column = UserRecipes.columns[o]
        #if value:
            #print("column: ",column)
            #print("User %i index %i: %i" %(i,o,value))
    

(arachide)olie              True
(herten)biefstukken         True
(houdbare) melk            False
(magere) shoarmareepjes     True
(olijf)olie                 True
                           ...  
zwarte tagliatelle          True
zwarte-peperkorrels         True
zwarteolijvenplakjes       False
zwartepeperkorrel          False
zwijnsteak                  True
Name: 0, Length: 4797, dtype: bool


## <font color=blue>Timing Debug info</font>

In [60]:
timingLabels = ["Loading column dataset", "Dictionary creation", "Generating profile data"]

for i in range(int(len(timingList)/2)):
    print("%s took %.2f seconds! "%(timingLabels[i],timingList[i*2+1]-timingList[i*2]))

Loading column dataset took 3.30 seconds! 
Dictionary creation took 0.06 seconds! 
Generating profile data took 71.74 seconds! 


In [5]:
UserIngredients

,(arachide)olie,(herten)biefstukken,(houdbare) melk,(magere) shoarmareepjes,(olijf)olie,(scharrel)braadworst,(wilde) perzik,(witte) quinoa,(wittewijn)azijn,(zonnebloem)olie,...,zwarte peper,zwarte peperkorrels,zwarte quinoa,zwarte rijst,zwarte sesamzaadje,zwarte tagliatelle,zwarte-peperkorrels,zwarteolijvenplakjes,zwartepeperkorrel,zwijnsteak
0,True,True,False,True,True,True,False,True,False,True,...,True,True,True,False,True,True,True,False,False,True
1,True,True,True,True,True,True,True,True,False,True,...,False,True,False,False,False,True,False,False,True,True
2,True,True,False,True,True,True,False,False,False,True,...,True,True,True,False,True,True,True,True,False,False
3,True,False,True,True,True,True,True,False,True,True,...,True,True,True,False,True,True,True,False,True,True
4,True,True,False,True,True,False,True,False,True,True,...,True,True,False,False,False,False,True,True,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,True,False,False,False,True,True,False,False,False,False,...,False,True,True,False,False,False,True,False,False,True
996,True,False,False,False,True,False,False,False,False,True,...,True,True,False,False,False,True,True,False,False,False
997,True,False,False,True,True,False,False,True,False,True,...,True,True,False,True,False,True,True,True,True,True
998,True,True,False,False,True,True,False,True,False,True,...,True,True,True,True,True,False,True,True,True,False


In [6]:
UserRecipes

,Andijviestamppot met gebakken salami,Libanese wraps met appel-koolsalade,Plaattaart met gegrilde groenten en witte kaas,Zalmspies met sojaglaze van de BBQ,&amp;Suus’ buik-weg-salade,'Burrito'-schotel met knapperige tortillapuntjes,'Foeyonghai' met bami,'Kapsalon' met&nbsp;vegetarische&nbsp;kipshoarma en salade,'Leeuwenkoppen' van gehakt en kool,'Marokkaanse' lamsbout,...,Zwarte-bonenstoof met feta,Zwartebonen-rijstsalade met avocado &amp; witte kaas,Zwartebonenchili met kip,Zwartebonenstoof met tofu en gordita's,Zweedse balletjes,Zweedse gehaktballetjes,Zweedse gehaktballetjes met room,Zweedse vegaballetjes met aardappel &amp; broccoli,Zwitserse kaasfondue,‘Nasi’ met Hollandse soepgroente
0,True,True,False,True,False,False,False,False,True,False,...,False,True,False,False,False,False,True,True,True,True
1,False,False,False,False,True,False,False,False,False,True,...,False,False,False,False,True,False,False,False,False,False
2,True,False,False,False,False,False,True,True,False,True,...,True,True,False,False,True,False,True,True,True,True
3,False,False,True,True,True,True,True,False,True,True,...,False,True,False,True,False,True,True,True,False,False
4,False,False,False,False,False,False,True,True,True,True,...,False,True,False,True,False,True,False,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
996,False,False,False,True,True,False,False,True,False,False,...,False,True,False,False,True,False,False,False,False,False
997,True,False,False,False,False,True,True,True,True,False,...,False,True,False,False,True,False,True,True,False,False
998,True,False,False,True,False,False,False,False,True,True,...,False,False,False,False,False,False,True,False,True,False


In [7]:
RecipeIngredients.set_index("recipe")

,(arachide)olie,(herten)biefstukken,(houdbare) melk,(magere) shoarmareepjes,(olijf)olie,(scharrel)braadworst,(wilde) perzik,(witte) quinoa,(wittewijn)azijn,(zonnebloem)olie,...,zwarte peper,zwarte peperkorrels,zwarte quinoa,zwarte rijst,zwarte sesamzaadje,zwarte tagliatelle,zwarte-peperkorrels,zwarteolijvenplakjes,zwartepeperkorrel,zwijnsteak
recipe,,,,,,,,,,,,,,,,,,,,,
Andijviestamppot met gebakken salami,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Libanese wraps met appel-koolsalade,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Plaattaart met gegrilde groenten en witte kaas,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zalmspies met sojaglaze van de BBQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&amp;Suus’ buik-weg-salade,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Zweedse gehaktballetjes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zweedse gehaktballetjes met room,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zweedse vegaballetjes met aardappel &amp; broccoli,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Boolean to Int

In [8]:
user_ingredients = UserIngredients*1
user_ingredients.sum()

(arachide)olie             1000
(herten)biefstukken         354
(houdbare) melk             348
(magere) shoarmareepjes     380
(olijf)olie                1000
                           ... 
zwarte tagliatelle          372
zwarte-peperkorrels         806
zwarteolijvenplakjes        346
zwartepeperkorrel           543
zwijnsteak                  362
Length: 4797, dtype: int64

In [9]:
user_recipes = UserRecipes*1
user_recipes.sum()

 Andijviestamppot met gebakken salami                 318
 Libanese wraps met appel-koolsalade                  354
 Plaattaart met gegrilde groenten en witte kaas       358
 Zalmspies met sojaglaze van de BBQ                   370
&amp;Suus’ buik-weg-salade                            333
                                                     ... 
Zweedse gehaktballetjes                               363
Zweedse gehaktballetjes met room                      334
Zweedse vegaballetjes met aardappel &amp; broccoli    341
Zwitserse kaasfondue                                  336
‘Nasi’ met Hollandse soepgroente                      328
Length: 4616, dtype: int64

# Dataset all 
ALL recipes with ALL ingredients and ALL tags

In [10]:
ingr = pd.read_csv('/data/foodboost/ingredients.csv', index_col=0)
nutr = pd.read_csv('/data/foodboost/nutritions.csv', index_col=0)
recp = pd.read_csv('/data/foodboost/recipes.csv', index_col=0)
tags = pd.read_csv('/data/foodboost/tags.csv', index_col=0)

ingr['all_ingredients'] = ingr[['recipe', 'ingredient']].groupby(['recipe'])['ingredient'].transform(lambda x: '|'.join(x))
recp_ingr = ingr.copy()
recp_ingr = recp_ingr[['recipe', 'all_ingredients']]
recp_ingr.drop_duplicates(inplace=True, ignore_index=True)

tags['all_tags'] = tags[['recipe', 'tag']].groupby(['recipe'])['tag'].transform(lambda x: '|'.join(x))
recp_tags = tags.copy()
recp_tags = recp_tags[['recipe','all_tags']]
recp_tags.drop_duplicates(inplace=True, ignore_index=True)

recp_all = pd.merge(recp_ingr, recp_tags[["recipe","all_tags"]] , how= 'outer', left_on='recipe', right_on='recipe')
recp_all

,recipe,all_ingredients,all_tags
0,Kruidnoten met choco-discodip,melkchocolade|kruidnoten|discodip,hollands|gebak|gebak|sinterklaas|sinterklaasavond
1,Kruidnoten in marsepein,blanke marsepein|ongezouten roomboter|kruidnot...,hollands|gebak|gebak|sinterklaas|sinterklaasavond
2,Kruidnoten met chocodips,pure chocolade|ongezouten roomboter|kruidnoten...,hollands|gebak|gebak|sinterklaas|sinterklaasavond
3,Pepernotentaart met marsepeinstrik,ongezouten roomboter|witte basterdsuiker|vanil...,oven|hollands|gebak|gebak|sinterklaas|sinterkl...
4,Perencake,middelgroot ei|Griekse yoghurt 10%|fijne krist...,vooraf te maken|oven|gebak|gebak
...,...,...,...
8701,Paddenstoelen en courgettegratin,witbrood|olijfolie|courgettes|knoflook|paddest...,oven|bijgerecht
8702,Peren-amandelcoupe,verse munt|halve peren op siroop|handpeer|vani...,mediterraan|nagerecht
8703,Ceviche met sint-jakobsschelpen,rode peper|verse gember|limoenen|rode grapefru...,voorgerecht
8704,Pittige truffels,rode peper|chocolade extra puur 72%|verse slag...,glutenvrij|oven|gebak|gebak|nagerecht


# Desired Columns
Make list of all recipes containing certain tag (example: 'chinees')

In [11]:
#kies een sublieme tag uit deze sublieme lijst
np.sort(tags['tag'].unique())

array(['5-ingrediënten', 'advertorial', 'amerikaans', 'argentijns',
       'aziatisch', 'bakken', 'barbecue', 'bijgerecht', 'biologisch',
       'blancheren', 'borrel', 'borrelhapje', 'brood/sandwiches',
       'brunch', 'budget', 'camping', 'caribisch', 'chileens', 'chinees',
       'colombia', 'couscous', 'curry', 'diner', 'drankje met alcohol',
       'drankje zonder alcohol', 'egyptisch', 'eiwitrijk', 'engels',
       'europees', 'fondue', 'frans', 'frituren', 'gebak', 'gezond',
       'glutenvrij', 'gourmet', 'grieks', 'grillen', 'halloween',
       'hollands', 'hoofdgerecht', 'in te vriezen', 'indiaas',
       'indonesisch', 'italiaans', 'jamaicaans', 'jamie oliver', 'japans',
       'kerst', 'keukenmachine', 'kidsfavoriet', 'kindergerecht',
       'kinderrecepten', 'koken', 'koningsdag', 'koreaans', 'lactosevrij',
       'lente', 'libanees', 'lunch', 'maaltijdsoep', 'marokkaans',
       'mealprep', 'mediterraan', 'mexicaans', 'midden-amerika',
       'midden-oosters', 'moederdag

In [15]:
recp_small = recp_all[recp_all['all_tags'].str.contains('chinees', na=False)]
list_small = list(recp_small['recipe'])
list_small

['Koe loe kai ',
 'Noedels met tofu, Chinese wokgroente en zoete chilisaus',
 'Crispy chili-olie ',
 'Open dumplings met garnalen in Siu Mai-stijl',
 'Congee',
 'Scallion pancakes',
 'Chinese chili-olie',
 'Chow mein',
 'Foe yong hai',
 'Tjap tjoy kip',
 'Babi pangang',
 'Bapao met gehakt ',
 'Chinese pickle ',
 'Wontons met krab en limoenblad ',
 'Wontons met varkensvlees en boerenkool',
 'Wontons met hoisineend en sojadip',
 'Dan dan-noedels met choisum',
 'Chinese chow mein met garnalen',
 'Kantonese kip met aspergebroccoli',
 'Foeyonghai',
 'Chinese balletjes in zoetzure saus',
 'Chinesekoolrolletjes met gehakt',
 'Shanghai paksoi met shiitake en bosui ',
 'Noedelsoep met Shanghai paksoi en kip',
 'Kantonese mie met varkenshaas',
 'Chinese omelet',
 'Roerbak van asperges en garnalen',
 'Zoetzuur varkensvlees met sesam-radijs',
 'Kip met paprika en chilipeper',
 'Biefstukreepjes met komijn en paprika',
 'Sesam-hoisinkip met broccoli',
 'Bami met paksoi en omelet',
 'Kung po beef',
 

In [16]:
user_recipes_small = user_recipes.loc[:,user_recipes.columns.str.contains('|'.join(list_small))]
user_recipes_small

,'Foeyonghai' met bami,'Leeuwenkoppen' van gehakt en kool,Babi pangang,Babi pangang express,Bami met paksoi en omelet,Bami met verse gember,Bami met zoete puntpaprika,Biefstukreepjes met komijn en paprika,Chinese balletjes in zoetzure saus,Chinese chow mein met garnalen,...,Koolsoep met varkensreepjes,Kung po beef,"Noedels met tofu, Chinese wokgroente en zoete chilisaus",Noedelsoep met Shanghai paksoi en kip,Oesterzwammen met gestoofde kip,Roerbak van asperges en garnalen,Sesam-hoisinkip met broccoli,Tjap tjoy kip,Zalmloempia's,Zoetzuur varkensvlees met sesam-radijs
0,0,1,1,0,0,0,1,0,0,1,...,1,1,0,1,1,0,0,1,0,1
1,0,0,0,1,0,0,0,1,0,1,...,0,0,0,0,0,0,0,1,1,0
2,1,0,1,1,1,1,1,0,1,1,...,1,0,0,1,0,1,0,0,0,1
3,1,1,0,0,0,0,1,1,0,0,...,0,1,0,1,1,1,0,1,1,1
4,1,1,0,0,0,1,1,1,0,1,...,1,0,1,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,1,1,0,0,0,0,0,0,...,0,0,0,1,0,1,1,0,0,1
996,0,0,1,0,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
997,1,1,0,1,1,0,1,0,0,0,...,0,0,1,0,0,0,1,0,1,1
998,0,1,1,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,1,0,1


# Logistic Regression

In [23]:
import numpy
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

y = user_recipes_small['Koe loe kai '].values
X = user_recipes_small.drop('Koe loe kai ', axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=36)

logr = LogisticRegression()
logr.fit(X,y)

y_pred = logr.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(logr.score(X_test, y_test))

[[114  19]
 [ 40  27]]
              precision    recall  f1-score   support

           0       0.74      0.86      0.79       133
           1       0.59      0.40      0.48        67

    accuracy                           0.70       200
   macro avg       0.66      0.63      0.64       200
weighted avg       0.69      0.70      0.69       200

0.705
